In [71]:
ts.j <- read.csv('c:/users/tom/pycharmprojects/nyu-research/data/series/alc-june.csv')
ts.s <- read.csv('c:/users/tom/pycharmprojects/nyu-research/data/series/alc-sept.csv')

In [4]:
head(ts.j)

Unnamed..0,Unnamed..1,casual,total
1,0,74,6005
1,1,45,3853
1,2,18,2415
1,3,15,1762
1,4,17,1648
1,5,20,2091


In [5]:
nsample <- sum(ts.j$total)

In [69]:
boot <- function(ts){
    new_ts <- ts
    new_ts$total <- 0
    total <- rmultinom(n=1, size=nsample, prob=ts$total)
    new_ts[,4] <- total
    for (i in 1:nrow(ts)){
        new_ts[i, 3] <- unname(rmultinom(n=1, size=total[i], prob=ts[i, c(3,4)]))[1]
    }
    new_ts
    }

In [73]:
boot.j <- boot(ts.j)
boot.s <- boot(ts.s)

In [77]:
head(ts.s)

Unnamed..0,Unnamed..1,casual,total
1,0,456,56662
1,1,258,32132
1,2,140,18445
1,3,98,11373
1,4,91,9846
1,5,101,12919


In [76]:
head(boot.s)

Unnamed..0,Unnamed..1,casual,total
1,0,471,54147
1,1,231,30784
1,2,125,17926
1,3,85,10781
1,4,87,9428
1,5,113,12401


In [78]:
source("C:/Users/Tom/PycharmProjects/nyu-research/ts.R")

Warning message:
"package 'zoo' was built under R version 3.3.3"
Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric

Warning message:
"package 'forecast' was built under R version 3.3.3"

In [90]:
library('TSclust')
library('factoextra')

Warning message:
"package 'TSclust' was built under R version 3.3.3"Loading required package: wmtsa
Warning message:
"package 'wmtsa' was built under R version 3.3.3"Loading required package: pdc
Warning message:
"package 'pdc' was built under R version 3.3.3"Loading required package: cluster
Warning message:
"package 'factoextra' was built under R version 3.3.3"Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.3.3"

In [80]:

preprocessing <- function(ts, thres=5000){
    df = time_index(data.frame(ts[,1:3]))
    df_total = time_index(data.frame(ts[,c(1,2,4)]))
    missing = df_total[, "tweets"] < thres
    print(which(missing))
    df_total[missing, "tweets"] <- NA
    df[missing, ] <- NA
    df_total = na.locf(df_total[, "tweets"])
    df_filled = (na.locf(df[, ncol(df)]))
    return(list(series = df_filled, total = df_total))
}

In [117]:
generate_plots <- function(boot.j, boot.s, simnum){
        
    june <- preprocessing(boot.j)
    june_cas = ts(june$series[33:(672+32)], frequency=24)
    june_tot = ts(june$total[33:(672+32)], frequency=24)
    june_perc <- june_cas/june_tot

    sept = preprocessing(boot.s)
    sept_cas = ts(sept$series[9:(672+8)], frequency=24)
    sept_tot = ts(sept$total[9:(672+8)], frequency=24)
    sept_perc <- sept_cas/sept_tot

    sept_days = split_by_day(sept_perc)
    colnames(sept_days) = c(1:28)
    june_days = split_by_day(june_perc)
    colnames(june_days) = c(102:129)
    combined = cbind(june_days, sept_days)

    phase_diss = dissimilarity(as.matrix(combined), phase_dist(1))
    weekdays = c(rep(c('Tu', 'W' ,'Th', 'F', 'Sa', 'Su', 'M'), 4), c('Tu', 'W' ,'Th', 'F', 'Sa', 'Su', 'M(L)'),
                 rep(c('Tu', 'W' ,'Th', 'F', 'Sa', 'Su', 'M'), 3))
    dist = phase_diss
    colnames(dist)=weekdays
    rownames(dist)=weekdays
    a=agnes(dist, diss=TRUE)

    phase_dend <-function(){
        p <- fviz_dend(a,  k=2, main='', ylab='Height', xlab='Day', las=1, cex=1.2, cex.lab=1.5, horiz=TRUE, label_col='black')
        # mtext( text='Day of Week', side=2, cex=1.5)
        p <- p + theme(axis.title=element_text(size=14,face="bold")) 
        p
    }
    phase_dend()
    ggsave(filename=paste('c:/users/tom/documents/research/bootstrap/phase-sim-', simnum, '.pdf', sep=""),device='pdf')
    colnames(combined)=weekdays
    dist = diss(combined, METHOD='EUCL')

    b = agnes(dist, diss=TRUE)
    euc_dend <-function(){
        p2 <- fviz_dend(b, k=4, k_colors=c("#ff6666", "#33cccc",'#66ff66', '#cc66ff'), main='', ylab='Height', xlab='Day', label_col='black', las=1, cex=1.2, cex.lab=1.5, horiz=TRUE)
        p2 <- p2 + theme(axis.title=element_text(size=14,face="bold"))
        p2
    }
    
    euc_dend()
    ggsave(filename=paste('c:/users/tom/documents/research/bootstrap/euc-sim-', simnum, '.pdf', sep=""),device='pdf')
}

In [118]:
for (i in 1:10){
    boot.j <- boot(ts.j)
    boot.s <- boot(ts.s)
    write.table(boot.j ,paste('c:/users/tom/documents/research/bootstrap/june-boot', i, '.csv', sep=""), sep=',')
    write.table(boot.j ,paste('c:/users/tom/documents/research/bootstrap/sept-boot', i, '.csv', sep=""), sep=',')
    generate_plots(boot.j, boot.s, simnum=i)

}

 [1]   2   3   4   5   6   7   8  26  27  28  29  30  31 721 722 723
 [1] 408 409 410 411 412 413 414 415 476 477 478 479 485 486 721 722 723 724 725


Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image


 [1]   2   3   4   5   6   7  26  27  28  29  30  31  32 721 722 723 724
 [1] 408 409 410 411 412 413 414 415 476 477 478 479 485 486 721 722 723 724 725


Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image


 [1]   2   3   4   5   6   7  26  27  28  29  30  31  32 721 722 723
 [1] 408 409 410 411 412 413 414 415 476 477 478 479 485 486 721 722 723 724 725


...

Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image


 [1]   2   3   4   5   6   7  26  27  28  29  30  31 721 722 723
 [1] 408 409 410 411 412 413 414 415 476 477 478 479 485 486 721 722 723 724 725


Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image


 [1]   2   3   4   5   6   7  26  27  28  29  30  31 721 722 723
 [1] 408 409 410 411 412 413 414 415 476 477 478 479 485 486 721 722 723 724 725


Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
